In [4]:
%pip install praw

import praw
import logging
import csv
import json
import time
import os


#suppress PRAW async warning by setting logging level to ERROR
logging.getLogger("praw").setLevel(logging.ERROR)

#set up PRAW with my created reddit app (commentdownloader) credentials
reddit = praw.Reddit(
    client_id='wK4F4t3ITVUZqDQRRhpXzQ',
    client_secret='uhJ8fb-lC-0RFjMRnR7EF-Cp81pnrw',
    user_agent='commentdownloader'
)




[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
#function to fetch Reddit comments and save to CSV in chunks
def fetch_comments_to_csv(subreddit_name, target_comments=2000000, batch_size=100):
    subreddit = reddit.subreddit(subreddit_name)
    total_comments = 0
    batch_counter = 1  # Track batch count for file naming
    
    while total_comments < target_comments:
        try:
            #open file for this batch
            file_path = f'reddit_comments_batch_{batch_counter}.csv'
            with open(file_path, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(['Post Title', 'Comment Body', 'Author'])
                
                #fetch a batch of submissions
                for submission in subreddit.new(limit=batch_size):
                    print(f"Processing Post: {submission.title}")
                    
                    submission.comments.replace_more(limit=None)
                    comments = submission.comments.list()
                    
                    #write comments to CSV
                    for comment in comments:
                        writer.writerow([submission.title, comment.body, str(comment.author)])
                    
                    total_comments += len(comments)
                    print(f"Collected {total_comments} comments so far.")
                    
                    #stop if target comments are reached
                    if total_comments >= target_comments:
                        break

            #save progress after each batch
            batch_counter += 1
            print(f"Batch {batch_counter-1} complete. File saved: {file_path}")

        except praw.exceptions.APIException as e:
            if e.error_type == 'RATELIMIT':
                print(f"Rate limit hit: {e}. Sleeping for 2.5 minutes...")
                time.sleep(150)  #sleep for 2.5 minutes
            else:
                print(f"API error occurred: {e}. Sleeping for 1 minute...")
                time.sleep(60)  #sleep for 1 minute
                
        except Exception as e:
            print(f"An unexpected error occurred: {e}. Sleeping for 1 minute...")
            time.sleep(60)  #sleep for 1 minute

    print("Comment collection complete.")

#fetch 2 million comments from the 'soccer' subreddit
fetch_comments_to_csv('soccer', target_comments=2000000)

Processing Post: Atletico hit with partial stadium closure after derby chaos
Collected 0 comments so far.
Processing Post: Charlotte FC [2]-1 Chicago Fire - Júnior Urso 30'
Collected 1 comments so far.
Processing Post: Charlotte FC 1-[1] Chicago Fire - Fabian Herbers 25'
Collected 2 comments so far.
Processing Post: Jhon Durán's Spanish post-match interview the CBS crew | Carragher and Micah Richards using Google Translate
Collected 4 comments so far.
Processing Post: Charlotte FC [1]-0 Chicago Fire - Karol Świderski 8'
Collected 7 comments so far.
Processing Post: Transfer Talk: Barcelona eyeing Neymar return
Collected 19 comments so far.
Processing Post: Match Thread: Columbus Crew vs Inter Miami CF | MLS
Collected 20 comments so far.
Processing Post: Atlético MG [2] x 1 Vasco - Paulinho 44'
Collected 24 comments so far.
Processing Post: Atlético MG [1] X 1 Vasco - Guilherme Arana 38' (Great goal)
Collected 26 comments so far.
Processing Post: Dani Carvajal throw-in vs LOSC Lille
Col